In [37]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import *
import math

In [38]:
#Input Pressure and Temperature values
P = float(input("Enter pressure in psia:")) # psia
T = float(input("Enter Temperature in Farenheit: ")) # Farenheit

print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )

For Pressure: 750.0 psia  and Temperature: -70.0 Farenheit


In [39]:
#Import Components Properties
component_properties = pd.read_excel("components.xlsx")
component_properties

,component,composition,Pci,Tci,wi,Mwi,Vci
0,N2,0.0200,493.1,227.49,0.040,28.013,0.0510
1,C1,0.8866,666.4,343.33,0.008,16.043,0.0988
2,C2,0.0492,706.5,549.92,0.098,30.070,0.0783
3,C3,0.0246,616.0,666.06,0.152,44.097,0.0727
4,nC4,0.0098,550.6,765.62,0.193,58.123,0.0703
5,nC5,0.0098,488.6,845.80,0.251,72.151,0.0675


In [40]:
#Import binary coefficients
d= pd.read_excel("binary.xlsx").to_numpy()
d

array([[0.    , 0.018 , 0.039 , 0.046 , 0.047 , 0.048 ],
       [0.018 , 0.    , 0.005 , 0.01  , 0.0145, 0.0182],
       [0.039 , 0.005 , 0.    , 0.0017, 0.0032, 0.0048],
       [0.046 , 0.01  , 0.0017, 0.    , 0.0012, 0.0024],
       [0.047 , 0.0145, 0.0032, 0.0012, 0.    , 0.0008],
       [0.048 , 0.0182, 0.0048, 0.0024, 0.0008, 0.    ]])

In [41]:
Pci = component_properties['Pci'] 
Tci = component_properties['Tci']

Pri= P/Pci
Tri= (T+459.67 )/Tci

print('Reduced Pressure,  Pri:\n', Pri)
print('Reduced Temperature, Tri:\n',Tri)

Reduced Pressure,  Pri:
 0    1.520990
1    1.125450
2    1.061571
3    1.217532
4    1.362150
5    1.534998
Name: Pci, dtype: float64
Reduced Temperature, Tri:
 0    1.712910
1    1.134972
2    0.708594
3    0.585037
4    0.508960
5    0.460712
Name: Tci, dtype: float64


In [42]:
wi = component_properties['wi'] #accentric factor

w_a = 0.45723553
w_b = 0.07779607

for i in wi:
    if i<0.49:
        mi = 0.3796+1.54226*wi-0.2699*wi**2
    else:
        mi = 0.379642+1.48503*wi-0.164423*wi**2+0.016667*wi**3
        
Ai =(w_a*(1+mi*(1-Tri**0.5))**2*(Pri/Tri**2)).replace(np.nan,0).to_numpy()
Bi = ((w_b*Pri)/Tri).to_numpy()
print('Ai:',Ai)
print('Bi:',Bi)

Ai: [0.17688642 0.37927951 1.13501871 2.12458098 3.41173111 5.09117878]
Bi: [0.06907951 0.07714339 0.1165492  0.16190289 0.20820877 0.2592007 ]


In [43]:
#Calculating A and B

c = component_properties['composition'].to_numpy()
n= len(c)

B =  np.sum(c * Bi)

A=0
for i in range (n):
    for j in range (n):
        A += (1-d[i,j])*(Ai[i]*Ai[j])**0.5*c[i]*c[j] 
        
print('A :', A)
print('B :', B)

A : 0.46468266556708404
B : 0.08407456358023657


In [44]:
#Calculating Z 

m1 = 1+2**0.5
m2 = 1-2**0.5
p=1
q=(m1+m2-1)*B-1
r=(A+m1*m2*B**2-(m1+m2)*B*(B+1))
s=-A*B-m1*m2*B**2*(B+1)

Z = np.roots([p, q, r, s])

print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )
print('                          A:', round(A,4))
print('                          B:', round(B,4))
print('                          Z:', Z)        
print('                      Z_max:', max(np.real(Z)))
print('                      Z_min:', min(np.real(Z)))


For Pressure: 750.0 psia  and Temperature: -70.0 Farenheit
                          A: 0.4647
                          B: 0.0841
                          Z: [0.47630776+0.j         0.21980884+0.13273547j 0.21980884-0.13273547j]
                      Z_max: 0.4763077590654529
                      Z_min: 0.21980883867715534


# Wilsons Correlation , K-Values 

In [45]:

Ki = (1/Pri)*((5.37*(1+wi)*(1-(1/Tri))).apply(math.exp))

Ki 

0    6.719563
1    1.691359
2    0.083362
3    0.010207
4    0.001519
5    0.000250
dtype: float64

# Composition of Liquid and Vapour phase 

In [46]:
g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

A = sum(c*(Ki-1))
B = sum(c*(Ki-1)/Ki)

v = A/(A - B) 

while abs(g_V(v))>0.0001:
    v = v - g_V(v)/dg_V(v)

l = 1 - v

print(f"Composition in vapor phase is: {v}")
print(f"Composition in liquid phase is: {l}")



Composition in vapor phase is: 0.8162445401238981
Composition in liquid phase is: 0.18375545987610187


# Calculation of A and B in liquid and vapour phase

In [47]:
Xi = c/(1 + v*(Ki-1))
Yi = Xi*Ki

def Aa(c):
    return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

Al = sum(Aa(Xi))
Av = sum(Aa(Yi))
Bl = sum(Xi*Bi)
Bv = sum(Yi*Bi)

print(f"A in Vapour phase {Av} and liquid phase {Al}")
print(f"B in Vapour phase {Bv} and liquid phase {Bl}")


A in Vapour phase 0.38411505086615205 and liquid phase 0.9281720994359663
B in Vapour phase 0.07771783257839684 and liquid phase 0.11231125911454326


In [48]:

# m1 = 1+2**0.5
# m2 = 1-2**0.5

# #Vapour Phase 
# p=1
# q=(m1+m2-1)*Bv-1
# r=(Av+m1*m2*Bv**2-(m1+m2)*Bv*(Bv+1))
# s=-Av*Bv-m1*m2*Bv**2*(Bv+1)

# Zv = np.roots([p, q, r, s])

# print('Vapour Phase compressibility factor, Zv:', np.real(Zv))

# Zv = max(Zv)
# print('Vapour Phase :', np.real(Zv))

# #Liquid phase 
# t=1
# u=(m1+m2-1)*Bl-1
# w=(Al+m1*m2*Bl**2-(m1+m2)*Bl*(Bl+1))
# x=-Al*Bl-m1*m2*Bl**2*(Bl+1)

# Zl = np.roots([t, u, w, x])

# for i in range(len(Zl)):
#     if np.isreal(Zl[i]):
#         print('Liquid Phase compressibility factor, Zl:', np.real(Zl[i]))




# Compresibility of Liquid and Vapour Phase #

In [49]:
Zl, Zv = symbols("Zl Zv")
f_Zl = Zl**3+(Bl-1)*Zl**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
f_Zv = Zv**3+(Bv-1)*Zv**2+(Av-3*(Bv)**2-2*Bv)*Zv-(Av*Bv-(Bv)**2-(Bv)**3)
Zl = solve(f_Zl,Zl)
Zv = solve(f_Zv,Zv)

# eliminate complex roots:
Zl = min([i for i in Zl if sp.im(i)==0]) # compressibility factor of liquid phase
Zv = max([i for i in Zv if sp.im(i)==0]) # compressibility factor of vapour phase
print(f"Vapour Phase compressibility factor, Zv: {Zv}")
print(f"Liquid Phase compressibility factor, Zl: {Zl}")

Vapour Phase compressibility factor, Zv: 0.655336658918527
Liquid Phase compressibility factor, Zl: 0.165103530488731


# Fugacity coefficients  and K-values correction # 

In [50]:
#fugacity of vapour phase  
phiv=(-math.log(Zv-Bv)+(Av/((m1-m2)*Bv)*(2*Ai*Yi/Av)-(Bi/Bv))*log(((Zv+m2*Bv)/(Zv+m1*Bv)))+Bi/Bv*(Zv-1)).apply(math.exp)

#fugacity of liquid phase  
phil=(-math.log(Zl-Bl)+(Al/((m1-m2)*Bl)*(2*Ai*Xi/Al)-(Bi/Bl))*log(((Zl+m2*Bl)/(Zl+m1*Bl)))+Bi/Bl*(Zl-1)).apply(math.exp)


# Equilibrium ratio:
K_i = (phil/phiv)

# print(f"fugacity co-eff of Vapour : {phiv}")
# print(f"fugacity co-eff of Liquid : {phil}")

## Equilibrium ratio anotherway
fv=phiv*Yi*P
fl=phil*Xi*P
K_ii = (Yi/Xi)*(fl/fv)
e_c=sum(((fl/fv)-1)**2)

print(f"Previous , Ki : {Ki.to_numpy()}")
print(f"===================================================================================")
print(f"Corrected, K_i : {K_i.to_numpy()}")
print(f"Corrected, K_ii: {K_ii.to_numpy()}")
print(f" e_c : {e_c}")

Previous , Ki : [6.71956334e+00 1.69135883e+00 8.33622357e-02 1.02070181e-02
 1.51868639e-03 2.50472233e-04]
Corrected, K_i : [15.24759507  7.3343337   3.23387821  2.53843609  6.63289471  4.0133977 ]
Corrected, K_ii: [15.24759507  7.3343337   3.23387821  2.53843609  6.63289471  4.0133977 ]
 e_c : 275844159.4936266


In [51]:
# iteration = 0.1
# K_i = Ki

# while iteration > 0.000000000000001:
#     Ki = K_i
#     g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
#     dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

#     A = sum(c*(Ki-1))
#     B = sum(c*(Ki-1)/Ki)

#     v = A/(A - B) 

#     while abs(g_V(v))>0.0001:
#         v = v - g_V(v)/dg_V(v)
#     l = 1 - v
    
#     Xi = c/(1 + v*(Ki-1))
#     Yi = Xi*Ki

#     def Aa(c):
#         return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

#     Al = sum(Aa(Xi))
#     Av = sum(Aa(Yi))
#     Bl = sum(Xi*Bi)
#     Bv = sum(Yi*Bi)
    
#     Zl, Zv = symbols("Zl Zv")
#     f_Zl = Zl**3+(Bl-1)*Zl**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
#     f_Zv = Zv**3+(Bv-1)*Zv**2+(Av-3*(Bv)**2-2*Bv)*Zv-(Av*Bv-(Bv)**2-(Bv)**3)
#     Zl = solve(f_Zl,Zl)
#     Zv = solve(f_Zv,Zv)

#     # eliminate complex roots:
#     Zl = min([i for i in Zl if sp.im(i)==0]) 
#     Zv = max([i for i in Zv if sp.im(i)==0])

#     phiv=(-math.log(Zv-Bv)+(Av/((m1-m2)*Bv)*(2*Ai*Yi/Av)-(Bi/Bv))*log(((Zv+m2*Bv)/(Zv+m1*Bv)))+Bi/Bv*(Zv-1)).apply(math.exp)
#     phil=(-math.log(Zl-Bl)+(Al/((m1-m2)*Bl)*(2*Ai*Xi/Al)-(Bi/Bl))*log(((Zl+m2*Bl)/(Zl+m1*Bl)))+Bi/Bl*(Zl-1)).apply(math.exp)

#     # Equilibrium ratio:
#     K_i = (phil/phiv)
#     iteration=sum(((fl/fv)-1)**2)
# iteration


# Density Calculation 

In [52]:
R= 10.731
Tk = T + 459.67 #in Rankine
Mwi= component_properties.Mwi
Mwl = sum(Xi*Mwi) # apparent molecular weight of liquid phase
Mwv = sum(Yi*Mwi) # apparent molecular weight of vapour phase
dl=(P*Mwl)/(R*Tk*Zl) #liquid density
dv=(P*Mwv)/(R*Tk*Zv) #vapor density

print(f"Density of Liquid : {dl}")
print(f"Density of Vapour : {dv}")


Density of Liquid : 30.0233057369810
Density of Vapour : 4.54218560520595


# Viscosity Calculation

In [53]:
for i in range (n):
    Vpc = sum(component_properties.Vci* Xi)
    Tpc = sum(Xi* Tci)
    Ppc =sum(Xi*Pci)

    Ei=5.4402*Tci**(1/6)/(Mwl**0.5*Pci**(2/3))

    if Tri[i]<=1.5:
        usi=34*10**(-5)*(Tri)**0.94/Ei
    elif Tri[i]>1.5:
        usi=(17.78*10**(-5)*(4.58*Tri-1.67)**0.625)/Ei
    us1 =Xi*usi*(Mwi**0.5)
    us2 =Xi*(Mwi**0.5)
    us = us1/us2


dr=Vpc*(dl/Mwl)
xv=3.5+986/Tk+0.01*Mwv
yv=2.4-0.2*xv
kv=(9.4+0.02*Mwv)*(Tk**1.5)/(209+19*Mwv+Tk)
Em=5.4402*Tpc**(1/6)/(Mwl**0.5*Ppc**(2/3))

#viscosity of vapour
visv=10**(-4)*kv*math.exp(xv*(dr/62.4)**yv)
print(f"Viscosity of Vapour : {visv}")

# viscosity of liquid
visl=sum(us)+(1/Em)*((0.1023+0.023364*dr+0.058533*dr**2-0.040758*dr**3+0.0093724*dr**4)**4-1*10**(-4)) 
print(f"Viscosity of Liquid : {visl}")


Viscosity of Vapour : 0.008217821871890595
Viscosity of Liquid : 0.0435165725876566


In [54]:
datatable = pd.DataFrame({"Components":component_properties.component,
                        "Molecular Weight":component_properties.Mwi,
                        "Composition [Given]":component_properties.composition,
                        "Mw of in Liquid": round(Xi*Mwi,4),
                        "Mw of in Vapor": round(Yi*Mwi,4),
                        "Composition in Liquid":round(Xi,4),
                        "Composition in Vapour":round(Yi,4), 
                        "Equilibrium Ratio":round(K_i,4)})
datatable

,Components,Molecular Weight,Composition [Given],Mw of in Liquid,Mw of in Vapor,Composition in Liquid,Composition in Vapour,Equilibrium Ratio
0,N2,28.013,0.0200,0.0988,0.6641,0.0035,0.0237,15.2476
1,C1,16.043,0.8866,9.0926,15.3789,0.5668,0.9586,7.3343
2,C2,30.070,0.0492,5.8755,0.4898,0.1954,0.0163,3.2339
3,C3,44.097,0.0246,5.6474,0.0576,0.1281,0.0013,2.5384
4,nC4,58.123,0.0098,3.0790,0.0047,0.0530,0.0001,6.6329
5,nC5,72.151,0.0098,3.8437,0.0010,0.0533,0.0000,4.0134


# Results

In [55]:

print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )
print(f"Composition of Vapor phase : {v} and Liquid Phase: {l}")
print(f"A in Vapour phase {Av} and Liquid phase {Al}")
print(f"B in Vapour phase {Bv} and Liquid phase {Bl}")
print(f"Compressibility factor,  Vapour phase , Zv: {Zv} and Liquid phase, Zl: {Zl} ")
print(f"Corrected, K_i : {K_i.to_numpy()}")
print(f"Density of   Vapour : {dv}   and Liquid : {dl}")
print(f"Viscosity of Vapour : {visv} and Liquid : {visl}")

For Pressure: 750.0 psia  and Temperature: -70.0 Farenheit
Composition of Vapor phase : 0.8162445401238981 and Liquid Phase: 0.18375545987610187
A in Vapour phase 0.38411505086615205 and Liquid phase 0.9281720994359663
B in Vapour phase 0.07771783257839684 and Liquid phase 0.11231125911454326
Compressibility factor,  Vapour phase , Zv: 0.655336658918527 and Liquid phase, Zl: 0.165103530488731 
Corrected, K_i : [15.24759507  7.3343337   3.23387821  2.53843609  6.63289471  4.0133977 ]
Density of   Vapour : 4.54218560520595   and Liquid : 30.0233057369810
Viscosity of Vapour : 0.008217821871890595 and Liquid : 0.0435165725876566
